In [7]:
import numpy as np
from numpy import kron as k
import scipy as sp
from scipy import linalg as lin

# Quantum Algorithims Tutorial

## Background 

### Computational basis

$| 0 \rangle = \begin{bmatrix}
                    1 \\
                    0
              \end{bmatrix}$
              

$| 1 \rangle = \begin{bmatrix}
                    0 \\
                    1
              \end{bmatrix}$
              
### Hadamard (plus/minus) basis

$| + \rangle = \frac{1}{\sqrt{2}}(| 0 \rangle + | 1 \rangle)$

$| - \rangle = \frac{1}{\sqrt{2}}(| 0 \rangle - | 1 \rangle)$

### 1-qubit gates
      
$ I = \begin{bmatrix}
        1 & 0 \\
        0 & 1
      \end{bmatrix}$   
      
$ X = \begin{bmatrix}
        0 & 1 \\
        1 & 0 
      \end{bmatrix}$
      
$ H = \frac{1}{\sqrt{2}} \begin{bmatrix}
                            1 & 1 \\
                            1 & -1
                         \end{bmatrix}$                     

$ Z = \begin{bmatrix}
        1 & 0 \\
        0 & -1 
      \end{bmatrix}$
      
### 2-qubit gates

$CNOT = \begin{bmatrix}
        1 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 1 \\
        0 & 0 & 1 & 0
        \end{bmatrix}$

In [27]:
# Computational basis states.
zero = np.array([1,0]);
one = np.array([0,1]);

# Hadamard (+/-) basis states.
plus = (1/np.sqrt(2))*(zero + one)
minus = (1/np.sqrt(2))*(zero - one)

# NOT (or bit flip) gate.
X = np.array([[0,1],[1,0]])

# Hadamard Gate.
H = (1/np.sqrt(2))*np.array([[1,1],[1,-1]])

# Pi phase shift (Pauli z) gate.
Z = np.array([[1,0],[0,-1]])

# 2 x 2 identity.
I = np.array([[1,0],[0,1]])

# 2-qubit controlled not gate.
CNOT = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])

# Kronecker product of an list of operators or states.
def kron(array):
    A = np.kron(array[0],array[1])
    if len(array) > 2:
        for element in array[2::]:
            A = np.kron(A,element)
    return A

# Nth state in the compuational basis.
def n_state(n,qubits):
    n_state = np.zeros(2**qubits)
    n_state[n] = 1
    return n_state

# 2^n x 2^n identity matrix.
def idn(qubits):
    return np.identity(2**qubits)

# State probability.
def prob(state):
    return np.multiply(state, state)

## Multiple qubit systems

Systems consisting of multiple qubits are formed by the Kroneckor (or tensor) product of single qubit states, eg:

$|00 \rangle = | 0 \rangle | 0 \rangle = | 0 \rangle \otimes | 0 \rangle = \begin{bmatrix} 1 \\ 0 \end{bmatrix} \otimes \begin{bmatrix} 1 \\ 0 \end{bmatrix} = \begin{bmatrix} 1 \\ 0 \\ 0 \\ 0 \end{bmatrix}$

To apply an $X$ gate to the first qubit of $| 00 \rangle$ and a $H$ gate to the second qubit we apply:

$(X \otimes H) | 00 \rangle = |1\rangle \frac{1}{\sqrt{2}} (|0 \rangle + | 1 \rangle) = |1 \rangle | + \rangle $

In [28]:
kron([X,H]) @ kron([zero,zero]) == kron([one,plus])

array([ True,  True,  True,  True])

Or to apply an $X$ gate to only the second qubit:

$(I \otimes X) | 00 \rangle = |01 \rangle$

In [29]:
kron([I,X]) @ kron([zero,zero]) == kron([zero, one])

array([ True,  True,  True,  True])

### Binary representation of multi-qubit states

Multi-qubit states my be abbreviated as $| i \rangle$, where $i$ is a decimal number, arrived at by interpreting the $0$s and $1$s in the braket as a binary string. For a 2-qubit system:

$| 00 \rangle \rightarrow | 0 \rangle$

$| 01 \rangle \rightarrow | 1 \rangle$

$| 10 \rangle \rightarrow | 2 \rangle$

$| 11 \rangle \rightarrow | 3 \rangle$

## Deutsch-Jozsa Algorithm

* Determine if a Boolean function of n Boolean inputs is even constant with O(1).
* Classically, this would require O(n) queries.
* Representation of a non-reversable function.

$U_f(|x \rangle | y \rangle) := | x \rangle| (y + f(x)) \text{ mod 2} \rangle $

* Phase kickback.

$ U_f | x \rangle | - \rangle = U_f| x \rangle \frac{1}{\sqrt{2}}(| 0 \rangle - |1 \rangle) = | x \rangle \frac{1}{\sqrt{2}}(| (f(x) + 0) \text { mod 2}) \rangle - | (f(x) + 1) \text { mod 2}) \rangle)$

$= (-1)^{f(x)}| x \rangle | - \rangle$

* Phase to bit flip.

$H Z H = X$

In [30]:
U_f0 = kron([I,I]);
U_f1 = kron([I,X]);
U_fx = CNOT;
U_fxbar = kron([I,X]) @ CNOT;

In [31]:
print((H @ Z @ H))
print(X)

[[-2.23711432e-17  1.00000000e+00]
 [ 1.00000000e+00 -2.23711432e-17]]
[[0 1]
 [1 0]]


In [32]:
psi_0 = kron([zero,one])
psi_1 = kron([H,H]) @ psi_0
psi_2 = U_fxbar @ psi_1
psi_3 = kron([H,idn(1)]) @ psi_2
np.multiply(psi_3,psi_3) 

array([0. , 0. , 0.5, 0.5])

## Bernstien-Vazirani Algorithm

* The quantum advantage of the Deutsch-Jozsa Algorithm disapears with a small error rate.
* BV algorithm shows quantum advantage even allowing for error.

### Multi-qubit controlled gates

In this example, the oracle is moddled by performing a bit flip on the last register if $a_i = 0$. This requires a CNOT gate between non-adjacent qubits. To arrive at a matrix representation of that operator let's first consider the algebraic representation of a controlled unitary operator:

$CU = | 0 \rangle \langle 0 | \otimes I + | 1 \rangle \langle 1 | \otimes U$

Where $I$ and $U$ are of the same dimension, $n$, and the control qubit is located in the first register. This follows the usual process whereby multi-qubit gates are 'built-up' as a series of tensor products. The new additions are the projection operators, $| 0 \rangle \langle 0 |$ and $| 1 \rangle \langle 1 | $, which map a single qubit to the $|0 \rangle$ and $| 1 \rangle$ states repectively.

As matrices, this takes the form (to arrive at this remember that $| 0 \rangle \langle 0 |$ is an outer product):

$CU = \begin{bmatrix}
        I_n & 0_n \\
        0_n & U_n \\
        \end{bmatrix}$
        
It is easy to check that this results in the desired behaviour if we consider a single qubit system:

$CU | 0 \rangle = \begin{bmatrix} 1 & 0 \\ 0 & U \end{bmatrix}  \begin{bmatrix} 1 \\ 0 \end{bmatrix} = \begin{bmatrix} 1 \\ 0 \end{bmatrix} = | 0 \rangle $

$CU | 1 \rangle = \begin{bmatrix} 1 & 0 \\ 0 & U \end{bmatrix}  \begin{bmatrix} 0 \\ 1 \end{bmatrix} = \begin{bmatrix} 0 \\ U \end{bmatrix} = U| 1 \rangle $

With the above in mind, for a 3-qubit system, a CNOT gate with the first qubit as the control and the third qubit as the target, $CX_{1,3}$, is:

$CX_{1,3} = | 0 \rangle \langle 0 | \otimes I_2\otimes I_2 + | 1 \rangle \langle 1 | \otimes I_2 \otimes X $

Or, for the same system, a CNOT gate with the second qubit as the control and the first as the target, $CX_{2,1}$:

$CX_{2,1} = I_2 \otimes | 0 \rangle \langle 0 | \otimes I_2 + X \otimes | 1 \rangle \langle 1 | \otimes I_2$

In [33]:
qubits = 4;
x = n_state(1,3);
a = [1,0,1];

In [15]:
Ua = (kron([np.outer(zero,zero),idn(3)]) + kron([np.outer(one,one),kron([idn(2),X])])) @ kron([I,I,CNOT])

In [16]:
psi_0 = kron([zero,zero,zero,one])

In [17]:
psi_1 = kron([H,H,H,H]) @ psi_0

In [18]:
psi_2 = Ua @ psi_1

In [19]:
psi_3 = kron([H,H,H,I]) @ psi_2

In [20]:
prob(psi_3)

array([0.00000000e+00, 0.00000000e+00, 1.13388168e-35, 1.13388168e-35,
       0.00000000e+00, 0.00000000e+00, 1.13388168e-35, 1.13388168e-35,
       0.00000000e+00, 0.00000000e+00, 5.00000000e-01, 5.00000000e-01,
       0.00000000e+00, 0.00000000e+00, 1.13388168e-35, 1.13388168e-35])

In [21]:
prob(1/np.sqrt(2)*(kron([one,zero,one,one]) + kron([one,zero,one,zero])))

array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. ,
       0. , 0. , 0. ])